# Regresión Lineal

In [202]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append('C:/Users/Fabiola/Documents/CUC/BigData/Programacion II/Optimizacion-y-Prediccion-de-Demanda-del-Transporte-Publico/src/eda')
import procesador_EDA
from procesador_EDA import ProcesadorEDA
sys.path.append('C:/Users/Fabiola/Documents/CUC/BigData/Programacion II/Optimizacion-y-Prediccion-de-Demanda-del-Transporte-Publico/src/datos')
from GestorDatos import GestorDatos

In [203]:
GestorDatos = GestorDatos()
df_pasajeros = GestorDatos.ObtencionPasajerosAdultosMayores()

2025-08-20 13:43:14,777 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-08-20 13:43:14,780 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-20 13:43:14,781 INFO sqlalchemy.engine.Engine select current_schema()
2025-08-20 13:43:14,784 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-20 13:43:14,786 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-08-20 13:43:14,786 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-08-20 13:43:14,788 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-20 13:43:14,790 INFO sqlalchemy.engine.Engine COMMIT
2025-08-20 13:43:14,790 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-20 13:43:14,794 INFO sqlalchemy.engine.Engine COMMIT
2025-08-20 13:43:14,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-20 13:43:14,799 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE 

Variable output Cantidad de pasajeros

In [205]:
df_pasajeros.head()
df_pasajeros.columns

Index(['id', 'annio', 'mes', 'totalpasajeros', 'heredia_sj', 'pavas_sanpedro',
       'sj_belen', 'cartago_sj', 'alaj_her'],
      dtype='object')

In [206]:
df_x = df_pasajeros[['annio','mes','heredia_sj','mes','pavas_sanpedro','sj_belen','cartago_sj','alaj_her']]
df_x

,annio,mes,heredia_sj,mes,pavas_sanpedro,sj_belen,cartago_sj,alaj_her
0,2020,Diciembre,876.0,Diciembre,126.0,297,538.0,39.0
1,2020,Noviembre,932.0,Noviembre,124.0,119,519.0,56.0
2,2020,Octubre,889.0,Octubre,388.0,53,532.0,127.0
3,2020,Setiembre,608.0,Setiembre,227.0,49,499.0,121.0
4,2020,Agosto,457.0,Agosto,44.0,65,325.0,205.0
...,...,...,...,...,...,...,...,...
103,2012,Mayo,2172.0,Mayo,41.0,467,NaN,NaN
104,2012,Abril,1143.0,Abril,50.0,271,NaN,NaN
105,2012,Marzo,1679.0,Marzo,77.0,384,NaN,NaN
106,2012,Febrero,777.0,Febrero,84.0,50,NaN,NaN


In [207]:
y = df_pasajeros['totalpasajeros']

In [208]:
pos_nan = ~y.isna()
df_x = df_pasajeros.loc[pos_nan, ['annio','mes','heredia_sj','mes','pavas_sanpedro','sj_belen','cartago_sj','alaj_her']]
y = y[pos_nan]

df_x = pd.get_dummies(df_x, drop_first=True)

In [209]:
df_x.head()

,annio,heredia_sj,pavas_sanpedro,sj_belen,cartago_sj,alaj_her,mes_Agosto,mes_Diciembre,mes_Enero,mes_Febrero,...,mes_Diciembre,mes_Enero,mes_Febrero,mes_Julio,mes_Junio,mes_Marzo,mes_Mayo,mes_Noviembre,mes_Octubre,mes_Setiembre
0,2020,876.0,126.0,297,538.0,39.0,False,True,False,False,...,True,False,False,False,False,False,False,False,False,False
1,2020,932.0,124.0,119,519.0,56.0,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2020,889.0,388.0,53,532.0,127.0,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,2020,608.0,227.0,49,499.0,121.0,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,2020,457.0,44.0,65,325.0,205.0,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [210]:
imputacion = SimpleImputer(strategy='mean')
x = imputacion.fit_transform(df_x.values)

In [211]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [212]:
regresion =  RandomForestRegressor(n_estimators=200, random_state=42)

In [213]:
regresion.fit(x_train, y_train)

,n_estimators,200
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## Verificar la precisión

In [214]:
cross_validation = cross_val_score(regresion, x_train, y_train, cv=5)

In [215]:
print(cross_validation)

[0.76859599 0.95176129 0.90606047 0.88882342 0.90571313]


In [216]:
print("Average 5-Fold CV Score: {}".format(np.mean(cross_validation)))

Average 5-Fold CV Score: 0.8841908602864954


In [217]:
y_pred = regresion.predict(x_test)

In [218]:
y_pred

array([2968.89 , 8227.5  , 1272.85 , 1993.605, 7069.22 , 6377.915,
       8066.21 , 9784.41 , 1812.925, 6306.62 , 2914.285, 2621.995,
       2242.6  , 2666.72 , 8146.455, 7438.085, 6082.59 , 3412.235,
       1550.52 , 6359.575, 7302.18 , 5791.585])

In [219]:
print("R^2: {}".format(regresion.score(x_test, y_test)))

R^2: 0.8971890359931631


In [94]:
x_test

array([[12.,  0.,  1.,  0.,  0.,  0.],
       [ 2.,  0.,  1.,  0.,  0.,  0.],
       [11.,  0.,  1.,  0.,  0.,  0.],
       ...,
       [ 7.,  1.,  0.,  0.,  0.,  0.],
       [ 6.,  0.,  0.,  0.,  0.,  1.],
       [ 9.,  0.,  0.,  0.,  0.,  1.]], shape=(3358, 6))

In [220]:
y_test

77      3756
10      8036
4       1096
83      3716
62      7015
67      6810
30      8516
45     12207
95      1510
11      4412
81      4002
79      3703
0       1876
80      3034
18      7996
69      7256
56      5618
76      3533
106      911
42      6647
12      6908
36      5560
Name: totalpasajeros, dtype: int64

In [221]:
x_test.shape

(22, 28)

In [222]:
y_test.shape

(22,)